In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble, cross_validation, metrics
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline

In [2]:
import matplotlib.pyplot as plt
 
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)
 
plt.style.use('ggplot')

In [3]:
dfTrain = pd.read_csv('../../data/train/train.csv')
dfTest = pd.read_csv('../../data/test/test.csv')

In [4]:
dfTrain.head(1)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,...,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
dfTest.head(1)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,...,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
dfTrain.shape

(42000, 785)

In [7]:
X_train = dfTrain.drop(['label'], axis=1)
X_test = dfTest
y_train = dfTrain.label

In [8]:
def cross_val_predict(clf, X, y):
    kf =  cross_validation.KFold(len(X), n_folds=5, shuffle=True)
  
    preds = np.zeros(len(X))
    preds_probas = np.zeros((len(X), len(y.value_counts())))
    for train_index, test_index in kf:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        clf.fit(X_train, y_train)
        preds[test_index] = clf.predict(X_test)
        preds_probas[test_index] = clf.predict_proba(X_test)
    return (preds_probas, preds)

#Modèles dummy

##ExtraTree basique

In [9]:
clf = ensemble.ExtraTreesClassifier(n_estimators = 50, n_jobs=3, max_depth=20)
(preds_probas, preds) = cross_val_predict(clf, X_train, y_train)

In [10]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.98      0.99      0.98      4132
          1       0.98      0.99      0.98      4684
          2       0.96      0.96      0.96      4177
          3       0.96      0.95      0.95      4351
          4       0.96      0.96      0.96      4072
          5       0.97      0.96      0.96      3795
          6       0.97      0.98      0.98      4137
          7       0.97      0.96      0.96      4401
          8       0.95      0.95      0.95      4063
          9       0.94      0.95      0.94      4188

avg / total       0.96      0.96      0.96     42000

[[4074    0    3    4    3    4   22    0   19    3]
 [   0 4617   17   16    8    3    5   10    6    2]
 [  19   10 4012   20   22    2   15   40   29    8]
 [   4    5   61 4127    4   50    4   29   47   20]
 [   8    6    6    2 3925    0   17   10   13   85]
 [  14    5    5   49   10 3627   27    8   29   21]
 [  20    7    3    0   11   22 4063    0  

In [16]:
clf.fit(X_train, y_train)
p = clf.predict(X_test)
pp = pd.DataFrame(p, columns=['Label'])
pp['ImageId'] = index=dfTest.index+1
pp.set_index('ImageId', inplace=True)
pp.to_csv('../../result/digits.csv')

In [37]:
clf = ensemble.ExtraTreesClassifier(n_estimators = 500, n_jobs=3, max_depth=20, max_features=500)
(preds_probas, preds) = cross_val_predict(clf, X_train, y_train)

In [38]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.98      0.99      0.98      4132
          1       0.98      0.99      0.98      4684
          2       0.96      0.96      0.96      4177
          3       0.97      0.95      0.96      4351
          4       0.97      0.97      0.97      4072
          5       0.97      0.96      0.96      3795
          6       0.97      0.98      0.98      4137
          7       0.97      0.97      0.97      4401
          8       0.96      0.95      0.96      4063
          9       0.95      0.95      0.95      4188

avg / total       0.97      0.97      0.97     42000

[[4075    0    6    2    3    5   13    2   21    5]
 [   1 4626   17   15    3    4    3    4    9    2]
 [  15   11 4027   21   21    2   17   36   19    8]
 [   9    5   58 4125    6   47    7   32   39   23]
 [   7    7    4    0 3931    1   22    6    7   87]
 [  13    8    3   33    5 3657   20    7   31   18]
 [  24    6    3    0   14   23 4046    0  

In [45]:
clf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs=3, max_depth=10)
(preds_probas, preds) = cross_val_predict(clf, X_train, y_train)

In [46]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.97      0.98      0.98      4132
          1       0.96      0.98      0.97      4684
          2       0.94      0.94      0.94      4177
          3       0.94      0.92      0.93      4351
          4       0.95      0.94      0.94      4072
          5       0.96      0.93      0.94      3795
          6       0.96      0.97      0.97      4137
          7       0.95      0.94      0.94      4401
          8       0.93      0.93      0.93      4063
          9       0.89      0.92      0.91      4188

avg / total       0.95      0.95      0.95     42000

[[4048    0    6    2    3    5   22    1   41    4]
 [   0 4596   23   16    9    6   11   11    8    4]
 [  18   18 3938   30   37    1   29   63   32   11]
 [   9   23   82 3995   10   71   10   46   64   41]
 [   7    7    9    1 3821    3   28    9   17  170]
 [  26   25    7   67    9 3542   40    8   34   37]
 [  22   12    3    0   14   39 4029    0  

#Avec une PCA préliminaire

In [14]:
from sklearn.decomposition import PCA, SparsePCA

In [50]:
X.shape

(42000, 784)

In [51]:
pca = PCA(n_components=X.shape[1]/10)
X_train_pca = pca.fit_transform(X_train)

In [55]:
clf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs=3, max_depth=10)
(preds_probas, preds) = cross_val_predict(clf, pd.DataFrame(X_train_pca), y_train)

In [56]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.96      0.96      0.96      4132
          1       0.96      0.97      0.97      4684
          2       0.92      0.90      0.91      4177
          3       0.86      0.90      0.88      4351
          4       0.92      0.91      0.91      4072
          5       0.91      0.89      0.90      3795
          6       0.94      0.96      0.95      4137
          7       0.94      0.93      0.93      4401
          8       0.87      0.87      0.87      4063
          9       0.89      0.88      0.89      4188

avg / total       0.92      0.92      0.92     42000

[[3974    0   16   28    6   17   51    8   25    7]
 [   1 4562   25   32    2   16   12    5   27    2]
 [  26   17 3762   82   54    7   27   39  147   16]
 [  10   16   82 3906    5   92   24   45  127   44]
 [   5   31   37   12 3711    9   44   14   42  167]
 [  27    4   22  172   40 3374   61   11   39   45]
 [  38    8   28    3   16   58 3979    0  

### Avec un scaler ça ira mieux

In [13]:
from sklearn import preprocessing

In [58]:
pca = PCA(n_components=X.shape[1]/10)
X_train_pca = pca.fit_transform(preprocessing.scale(X_train))

/home/issam/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:332: UserWarning: The scale function assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))
/home/issam/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:145: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  Xr -= mean_
/home/issam/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:147: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  Xr /= std_


In [59]:
clf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs=3, max_depth=10)
(preds_probas, preds) = cross_val_predict(clf, pd.DataFrame(X_train_pca), y_train)

In [60]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.93      0.95      0.94      4132
          1       0.95      0.97      0.96      4684
          2       0.86      0.85      0.85      4177
          3       0.81      0.83      0.82      4351
          4       0.87      0.87      0.87      4072
          5       0.82      0.77      0.79      3795
          6       0.92      0.93      0.93      4137
          7       0.88      0.88      0.88      4401
          8       0.81      0.77      0.79      4063
          9       0.82      0.84      0.83      4188

avg / total       0.87      0.87      0.87     42000

[[3942    0   29   33    7   53   47    8    7    6]
 [   1 4533   38   14   17    7   32    6   25   11]
 [  65   16 3538  191   57   16   80   42  153   19]
 [  26   25  145 3623   16  104   14   76  236   86]
 [   6   54   98    6 3555   47   18   38   28  222]
 [  73    3   56  302   53 2925  100   26  174   83]
 [  65    9   49    7   41   97 3837    2  

# Testons des modèles linaires

In [12]:
from sklearn import linear_model

In [68]:
clf = linear_model.SGDClassifier(loss='log')
(preds_probas, preds) = cross_val_predict(clf, pd.DataFrame(X_train), y_train)

In [69]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.92      0.96      0.94      4132
          1       0.91      0.97      0.94      4684
          2       0.90      0.83      0.86      4177
          3       0.90      0.81      0.86      4351
          4       0.87      0.91      0.89      4072
          5       0.80      0.81      0.81      3795
          6       0.95      0.91      0.93      4137
          7       0.91      0.85      0.88      4401
          8       0.79      0.81      0.80      4063
          9       0.80      0.87      0.83      4188

avg / total       0.88      0.87      0.87     42000

[[3968    1   33    8   12   27   22    8   43   10]
 [   0 4546   23    8    5   18    7    3   66    8]
 [  42  117 3465   48   89   42   34   81  211   48]
 [  35   57  157 3539   14  223   15   60  171   80]
 [  13   20   21   11 3708   21   11   14   44  209]
 [  70   20   34  132   77 3083   63   20  231   65]
 [  52   28   48    9   67  131 3748    8  

In [70]:
clf = linear_model.SGDClassifier(loss='modified_huber')
(preds_probas, preds) = cross_val_predict(clf, pd.DataFrame(X_train), y_train)

In [71]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.95      0.95      0.95      4132
          1       0.89      0.98      0.93      4684
          2       0.88      0.86      0.87      4177
          3       0.89      0.81      0.85      4351
          4       0.86      0.90      0.88      4072
          5       0.71      0.82      0.76      3795
          6       0.96      0.90      0.93      4137
          7       0.90      0.86      0.88      4401
          8       0.83      0.70      0.76      4063
          9       0.80      0.86      0.83      4188

avg / total       0.87      0.87      0.87     42000

[[3944    0   30    9   13   61   28    6   31   10]
 [   0 4580   22   10    1   27    2    5   29    8]
 [  19  107 3574   50   83   62   29   83  139   31]
 [  12   65  182 3518   17  319   10   54  108   66]
 [   7   29   21   11 3685   36   16   22   17  228]
 [  52   27   46  153   87 3103   54   23  188   62]
 [  39   42   66    8   54  159 3731   10  

In [16]:
clf = linear_model.LogisticRegression()
(preds_probas, preds) = cross_val_predict(clf, pd.DataFrame(preprocessing.scale(X_train)), y_train)

/home/issam/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:145: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  Xr -= mean_
/home/issam/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:147: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  Xr /= std_


In [18]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.95      0.97      0.96      4132
          1       0.94      0.97      0.96      4684
          2       0.90      0.88      0.89      4177
          3       0.88      0.87      0.88      4351
          4       0.90      0.91      0.91      4072
          5       0.87      0.85      0.86      3795
          6       0.94      0.95      0.94      4137
          7       0.91      0.92      0.91      4401
          8       0.87      0.84      0.85      4063
          9       0.87      0.86      0.87      4188

avg / total       0.90      0.90      0.90     42000

[[3993    0   15   15    8   36   30    4   24    7]
 [   0 4562   29   13    5   13    9    9   41    3]
 [  42   58 3655   80   60   25   65   68  107   17]
 [  16   21  128 3798   11  154   21   48  100   54]
 [  14   25   35   12 3724    7   24   28   44  159]
 [  43   25   30  152   57 3240   67   20  104   57]
 [  37   12   39    4   30   58 3922    3  

# La RBM à la rescousse

In [28]:
rbm = BernoulliRBM(n_components=100)
clf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs=3, max_depth=10)
pipe = Pipeline(steps=[('rbm', rbm), ('rf', clf)])
(preds_probas, preds) = cross_val_predict(clf, X_train, y_train)

In [29]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.97      0.98      0.97      4132
          1       0.96      0.98      0.97      4684
          2       0.95      0.94      0.94      4177
          3       0.94      0.92      0.93      4351
          4       0.95      0.94      0.94      4072
          5       0.95      0.93      0.94      3795
          6       0.96      0.97      0.97      4137
          7       0.95      0.93      0.94      4401
          8       0.93      0.93      0.93      4063
          9       0.89      0.92      0.91      4188

avg / total       0.95      0.95      0.95     42000

[[4045    0    5    3    4    7   20    2   43    3]
 [   0 4594   26   15    9    6   11   11    8    4]
 [  22   17 3930   30   39    3   31   59   30   16]
 [   9   26   79 3993    9   77   11   44   61   42]
 [   7    6   10    0 3829    3   26   10   20  161]
 [  24   22    5   72    9 3547   42    6   29   39]
 [  23   13    2    0   16   43 4021    1  

In [33]:
rbm = BernoulliRBM(n_components=100)
lgr = linear_model.LogisticRegression()
pipe = Pipeline(steps=[('rbm', rbm), ('logistic', lgr)])
(preds_probas, preds) = cross_val_predict(clf, X_train, y_train)

In [34]:
print metrics.classification_report(y_train, preds)
print metrics.confusion_matrix(y_train, preds)

             precision    recall  f1-score   support

          0       0.97      0.98      0.98      4132
          1       0.96      0.98      0.97      4684
          2       0.94      0.94      0.94      4177
          3       0.94      0.92      0.93      4351
          4       0.95      0.94      0.94      4072
          5       0.95      0.93      0.94      3795
          6       0.96      0.97      0.97      4137
          7       0.96      0.94      0.95      4401
          8       0.93      0.92      0.93      4063
          9       0.89      0.92      0.91      4188

avg / total       0.95      0.95      0.95     42000

[[4053    0    4    2    3    7   21    0   38    4]
 [   0 4593   26   17    9    5   10   11    8    5]
 [  21   17 3932   33   39    1   29   59   31   15]
 [  11   22   85 3996    7   71   12   40   62   45]
 [   7    7    7    0 3828    3   25    9   18  168]
 [  25   24    6   71    8 3543   40    8   36   34]
 [  26   11    4    0   14   41 4020    1  